In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import os
from skimage import io
from PIL import Image
import csv
import pandas as pd
from keras.models import load_model

In [ ]:
# 读取训练集图片
filenames=os.listdir('D:/train/')

filenames.sort(key=lambda x:int(x[:-4])) 

def read_image1(filename):
    img = Image.open(('D:/train/')+filename).convert('RGB')
    return np.array(img)

x_train = []

for i in filenames:  
    x_train.append(read_image1(i))

x_train = np.array(x_train)

In [ ]:
# 导入训练集标签,y_train是训练集的标签
y_train = []
csv_file = csv.reader(open('D:/train.csv'))
for content in csv_file:
    content = list(map(float, content))
    y_train.append(content)
y_train = np.array(y_train)

In [ ]:
# 读取测试集图片
filenames=os.listdir('D:/test/')

filenames.sort(key=lambda x:int(x[:-4])) 

def read_image1(filename):
    img = Image.open(('D:/test/')+filename).convert('RGB')
    return np.array(img)

x_test = []

for i in filenames:  
    x_test.append(read_image1(i))

x_test = np.array(x_test)

In [ ]:
# 导入测试集标签,y_test是测试集的标签
y_test = []
csv_file = csv.reader(open('D:/test.csv'))
for content in csv_file:
    content = list(map(float, content))
    y_test.append(content)
y_test = np.array(y_test)

In [ ]:
# 将特征点从0~255转换成0~1提高特征提取精度
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_normalize = x_train / 255 
x_test_normalize = x_test / 255

In [ ]:
# 模型架构
from keras.models import Sequential
from keras.layers import  Activation, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), strides=(1, 1), input_shape=(336, 336, 3), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(keras.layers.BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
#设置模型参数
from keras.optimizers import Adam
model.compile(optimizer =Adam(lr = 1e-4),
              loss = 'mean_absolute_error')

In [ ]:
#训练
history = model.fit(x_train_normalize, y_train, 
                    validation_split = 0.25, epochs = 300, batch_size = 64, verbose = 2)

In [ ]:
model.evaluate(x_test_normalize, y_test)

In [ ]:
pred = model.predict(x_test_normalize)
print(pred)

In [ ]:
pred_1 = model.predict(x_train_normalize)
print(pred_1)

In [ ]:
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])  
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.xlabel('Epoch')
    plt.ylabel(train)  
    plt.legend(['train', 'validation'], loc = 'best') 
    
show_train_history(history, 'loss', 'val_loss') 

In [ ]:
#测试集散点图
x=np.linspace(350,600,1000)
y=x
plt.plot(x,y,color='blue',linestyle='--')
plt.title('Test vs Prediction')
plt.xlabel('Test')
plt.ylabel('Prediction')
plt.scatter(y_test,pred,s=9.,color='red')

In [ ]:
#训练集散点图
x=np.linspace(350,600,1000)
y=x
plt.plot(x,y,color='blue',linestyle='--')
plt.title('Train vs Prediction')
plt.xlabel('Train')
plt.ylabel('Prediction')
plt.scatter(y_train,pred_1,s=9.,color='red')

In [ ]:
# R2&MAE计算公式
def r2_mae(y_act, y_pre):
    error = []
    for ap in range(len(y_act)):
        error.append(y_act[ap] - y_pre[ap])
        '''
        print("Errors: ", Error)
        print(Error)
        '''
    squared_error = []
    abs_error = []
    for ae in error:
        squared_error.append(ae * ae)  
        abs_error.append(abs(ae))  
    target_deviation = []
    target_mean = sum(y_act) / len(y_act) 
    for ac in y_act:
        target_deviation.append((ac - target_mean) * (ac - target_mean))
    r1 = []
    for n in range(len(y_act)):
        r1.append(y_act[n] * y_pre[n])
    r2 = np.square(len(y_act) * sum(r1) - sum(y_act) * sum(y_pre))
    r3 = []
    for n in range(len(y_act)):
        r3.append(y_pre[n] * y_pre[n])
    r4 = len(y_act) * sum(r3) - sum(y_pre) * sum(y_pre)
    r5 = []
    for n in range(len(y_act)):
        r5.append(y_act[n] * y_act[n])
    r6 = len(y_act) * sum(r5) - sum(y_act) * sum(y_act)
    rr2 = r2 / (r4 * r6)
    mae = sum(abs_error) / len(abs_error)
    return rr2, mae

In [ ]:
r2_mae(y_test,pred)

In [ ]:
r2_mae(y_train,pred_1)

In [ ]:
#保存模型
model.save('D:/DP.h5')